In [1]:
import os
import glob
import pandas as pd
import tensorflow as tf
from absl import logging
from modules import components, pipeline
from modules.utils import merge_dataset
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

c:\Users\zis\anaconda3\envs\movies-rec\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
PIPELINE_NAME = "cbf_pipeline"

# pipeline inputs
DATA_ROOT = "data/merge"
TRANSFORM_MODULE_FILE = "modules/cbf_transform.py"
TUNER_MODULE_FILE = "modules/cbf_tuner.py"
TRAINER_MODULE_FILE = "modules/cbf_trainer.py"

# pipeline outputs
OUTPUT_BASE = "outputs"

serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")


In [3]:
components = components.init_components(
    data_dir=DATA_ROOT,
    transform_module=TRANSFORM_MODULE_FILE,
    tuner_module=TUNER_MODULE_FILE,
    trainer_module=TRAINER_MODULE_FILE,
    train_steps=1000,
    eval_steps=500,
    serving_model_dir=os.path.join(
        serving_model_dir, "cbf_model"),
    epochs=10,
)

pipeline = pipeline.init_pipeline(
    pipeline_root=pipeline_root,
    pipeline_name=PIPELINE_NAME,
    metadata_path=metadata_path,
    components=components,
)

BeamDagRunner().run(pipeline)

Trial 11 Complete [00h 00m 22s]
multi_objective: 3.331780433654785

Best multi_objective So Far: 2.6181695461273193
Total elapsed time: 00h 03m 31s

Search: Running Trial #12

Value             |Best Value So Far |Hyperparameter
232               |264               |dense_unit1
264               |8                 |dense_unit2
40                |40                |dense_unit3
1e-05             |0.001             |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 genres_xf (InputLayer)         [(None, 1)]          0           []                               
                                             